# Tutorials on using Meteor Package to solve quantum problems
- Simulation of Boson sampling
- Simulation of quantum circuit
- Simulation of variational quantum circuit

## Part 1: Simulation of Boson sampling

Pipeline for boson sampling simulation:
- Generating initial separable quantum state
- Generating an optical circuit from phase shifters and beam spliters
- Applying the optical circuit onto the quantum state and obtaining amplitudes 

In [1]:
include("../find_meteor.jl")

using Meteor.TensorNetwork
using Meteor.OpticalCircuit
using Meteor.OpticalCircuit.Utility

# Generating an initial state

L = 6
N = div(L, 2)
d = N+1
depth = L

initial_state = [0 for _ in 1:L]
initial_state[1:N] .= 1

# random basis generate 10 random basis with L modes, N particles, and a maximum of d-1 particles per mode
basis = randombasis(L, d, N, 10)

10-element Array{Array{Int64,1},1}:
 [2, 0, 1, 0, 0, 0]
 [0, 0, 2, 0, 1, 0]
 [0, 0, 0, 2, 0, 1]
 [0, 0, 0, 0, 3, 0]
 [0, 0, 1, 0, 0, 2]
 [1, 2, 0, 0, 0, 0]
 [0, 2, 0, 0, 1, 0]
 [0, 0, 2, 1, 0, 0]
 [3, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 1, 1]

In [2]:
# generating random bosonic circuit

using LinearAlgebra: I
function generate_boson_circuit_with_U(L::Int, N::Int, depth::Int)
    circuit = BosonCircuit(N)
    U = Matrix{ComplexF64}(I, L, L)
    for i = 1:L
        phi = rand(Float64)
        add_phase_shifter!(circuit, i, phi)
        U = phase_shifter_matrix(phi, L, i) * U
    end
    for d = 0:(depth-1)
        if d % 2 ==0
            for i = 1:2:(L-1)
                theta = rand(Float64)
                add_beam_spliter!(circuit, (i, i+1), theta)
                U = beam_spliter_matrix(theta, L, (i, i+1)) * U
            end
            for i = 1:2:(L-1)
                phi = rand(Float64)
                add_phase_shifter!(circuit, i, phi)
                U = phase_shifter_matrix(phi, L, i) * U
            end
        else
            for i = 2:2:(L-1)
                theta = rand(Float64)
                add_beam_spliter!(circuit, (i, i+1), theta)
                U = beam_spliter_matrix(theta, L, (i, i+1)) * U
            end
            for i = 2:2:(L-1)
                phi = rand(Float64)
                add_phase_shifter!(circuit, i, phi)
                U = phase_shifter_matrix(phi, L, i) * U
            end
        end
    end
    return circuit, U
end


circuit, U = generate_boson_circuit_with_U(L, N, depth)
println("number of gate operations $(length(circuit)).")

number of gate operations 36.


### Random optical circuit generated above ![BosonSampling](bosonsampling.png)

### First approach to compute amplitudes-evolving the state to get a final state, and then perform measurement

In [3]:
using LinearAlgebra: dot

state = complex(product_state(circuit, initial_state))
apply!(circuit, state)

# measure amplitude one by one
amps = [dot(state, product_state(circuit, item)) for item in basis]

10-element Array{Complex{Float64},1}:
   -0.08818423191756873 + 0.00649190683460827im
  -0.010101309401909184 + 0.06269494318354432im
  -0.013891847670323485 + 0.010526464598058118im
 -0.0026004628889715786 - 0.0015951968979177609im
  0.0035541355930629563 + 0.01126783015753214im
   0.028034548768339683 + 0.27410755832398737im
  -0.036370981975918426 + 0.009751224072243925im
    -0.0345930943164333 - 0.03800067214563224im
   0.000444391844120708 + 0.0003726239780160468im
  -0.008162281004749755 + 0.012050375168972685im

One can obtain additional information since the final state has been stored-entropy production example

In [4]:
# get the entropy from the final state
using Meteor.Tensor: entropy

etps = entropy.(svectors(state))

7-element Array{Float64,1}:
 -0.0
  0.3643836024659358
  1.0495096086240001
  1.3218098655614527
  0.8776694464200893
  0.24687489624218106
 -0.0

### Second approach to compute amplitudes-directly performing measurement, more efficiently if only a few number of amplitudes are to be computed

In [5]:
amps = amplitudes(circuit, initial_state, basis, scaling=1.)

10-element Array{Complex{Float64},1}:
   -0.08818423191756881 - 0.006491906834608374im
  -0.010101309401909158 - 0.06269494318354435im
  -0.013891847670323492 - 0.010526464598058128im
 -0.0026004628889715795 + 0.00159519689791776im
  0.0035541355930629667 - 0.011267830157532157im
   0.028034548768339784 - 0.27410755832398703im
   -0.03637098197591839 - 0.009751224072243955im
  -0.034593094316433246 + 0.038000672145632214im
  0.0004443918441206983 - 0.00037262397801604243im
   -0.00816228100474975 - 0.0120503751689727im

### Comparison with permanent

In [6]:
amp1s = ComplexF64[]
amp2s = ComplexF64[]
for item in basis
    amp1 = amplitude(circuit, initial_state, item, scaling=1.)
    
    m = select_submatrix(item, U, initial_state)
    amp2 = permanent(m)
    push!(amp1s, amp1)
    push!(amp2s, amp2)
end

println("largest error between two approaches $(maximum(abs.(amp1s-amp2s))).")

largest error between two approaches 2.2076699176994166e-16.


## Part 2: Simulation of quantum circuit

Pipeline for quantum circuit simulation:
- Generating initial separable quantum state
- Generating a quantum circuit from phase shifters and beam spliters
- Applying quantum measurements

### A trivial example-bell state

In [7]:
using Meteor.QuantumCircuit

state = statevector(2)

circuit = QCircuit([HGate(1), CNOTGate(1,2)])

apply!(circuit, state)

# measuring amplitudes
println("the amplitude on [0,0] is $(amplitude(state, [0,0], scaling=1.)).")
println("the amplitude on [0,1] is $(amplitude(state, [0,1], scaling=1.)).")
println("the amplitude on [1,0] is $(amplitude(state, [1,0], scaling=1.)).")
println("the amplitude on [1,1] is $(amplitude(state, [1,1], scaling=1.)).")

# measuring each qubit
i, prob = measure!(state, 1)
println("the probability to obtain state $i for the 1-th qubit is $prob.")

the amplitude on [0,0] is 0.7071067811865475 + 0.0im.
the amplitude on [0,1] is 0.0 + 0.0im.
the amplitude on [1,0] is 0.0 + 0.0im.
the amplitude on [1,1] is 0.7071067811865475 + 0.0im.
the probability to obtain state 1 for the 1-th qubit is 0.4999999999999999.


### Understand the QuantumCircuit module in depth
Different backend with almost the same interface
- Store the state vector in a brute-force way, exponential scaling $2^n$ with number of qubits, suitable for very deep circuit with number of qubits less than $36$ 
- Store the state vector using matrix product states (MPS), scaling $2n D^2$, where the **bond dimension** $D$ is related with the bipartition entropy, suitable for circuit which will only generate relativelly low entanglement, such as shallow quantum circuit
- Schodinger-Feymann simulator, splitting $n$ qubits into two parts with two brute-force simulator, suitable for quantum circuit which could be divided into two separate circuits by **cutting** only a few cross gates
- Projected entangled pair states (PEPS), used for quantum circuit on an arbitrary graph, with realtively low circuit depth

Example with two-dimensional random quantum circuit

### Introduction to Random Quantum Circuits
    
1. Finding a problem which is very hard to solve with the best supercomputer in the world, but can be efficiently solved with a quantum computer.
2. Random quantum circuit (**RQCs**) sampling problem is a standard problem used to demonstrate quantum supremacy in near term.
3. We propose to use Projected Entangled Pair States (**PEPS**) to study large-scale RQCs.

The random quantum circuit proposed in [Characterizing Quantum Supremacy in Near-Term Devices](arXiv:1608.00263v3)
![google circuit](GoogleCircuitH.png)

Rules for $L_h\times L_v$ circuit:
1. Apply a Hadamard gate to each qubit to initialize the qubits to a symmetric superposition.
2. Apply controlled-phase (CZ) gates alternating between $8$ configurations as above entangle neighbouring qubits.
3. Apply a randomly chosen gate ($T$, $\sqrt{X}$ or $\sqrt{Y}$) to each qubit on which the CZ gates has not just been applied, according to certain rules.
4. Repeat steps 2 and 3 to add layers of depth to the circuit.
5. Apply a final Hadamard gate to each qubit.

In [8]:
# Genertae an m * n random quantum circuit with depth d
function google_circuit(m::Int, n::Int, depth::Int)
    XYT = Dict(0=>sqrtX, 1=>sqrtY, 2=>T)
    Xoperated = zeros(Bool, m, n)
    Yoperated = zeros(Bool, m, n)
    Toperated = zeros(Bool, m, n)
    CZoperated = zeros(Bool, m, n)

    ControlZ = CZ
    config1 = []
    config2 = []
    config3 = []
    config4 = []
    config5 = []
    config6 = []
    config7 = []
    config8 = []
    config1sites = []
    config2sites = []
    config3sites = []
    config4sites = []
    config5sites = []
    config6sites = []
    config7sites = []
    config8sites = []
    toind(i,j) = linear_index((m, n), (i, j), false)

    # 1 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config1, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config1sites, (i, j))
            push!(config1sites, (i, j+1))
        end
    end

    # 2 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config2, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config2sites, (i, j))
            push!(config2sites, (i, j+1))
        end
    end

    # 5 configuration
    for jj in 1:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config5, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config5sites, (i, j))
            push!(config5sites, (i, j+1))
        end
    end

    # 6 configuration
    for jj in 1:2:(n-2)
        for ii in (((div(jj,2)))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config6, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config6sites, (i, j))
            push!(config6sites, (i, j+1))
        end
    end

    # 3 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config3, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config3sites, (i, j))
            push!(config3sites, (i+1, j))
        end
    end

    # 4 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config4, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config4sites, (i, j))
            push!(config4sites, (i+1, j))
        end
    end

    # 7 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config7, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config7sites, (i, j))
            push!(config7sites, (i+1, j))
        end
    end

    # 8 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config8, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config8sites, (i, j))
            push!(config8sites, (i+1, j))
        end
    end

    configs=[(config1, config1sites), (config2, config2sites), (config3, config3sites),
        (config4, config4sites), (config5, config5sites), (config6, config6sites),
        (config7, config7sites), (config8, config8sites)]

    function __generate_one_body_gates()
        one_body_gates = []
        tmpXoperated = zeros(Bool, m, n)
        tmpYoperated = zeros(Bool, m, n)
        for i in 1:m
            for j in 1:n
                # Place a gate at qubit q only if this qubit is
                # occupied by a CZ gate in the previous cycle
                if CZoperated[i, j]
                    # Place a T gate at qubit q if there are no single
                    # qubit gates in the previous cycles at qubit q except
                    # for the initial cycle of Hadamard gates
                    if !Toperated[i, j]
                        push!(one_body_gates, (toind(i, j), XYT[2]))
                        Toperated[i, j] = true
                    else
                        # Any gate at qubit q should be different from
                        # the gate at qubit q in the previous cycle.
                        if tmpXoperated[i, j]
                            push!(one_body_gates, (toind(i, j), XYT[1]))
                            tmpYoperated[i, j] = true
                        else
                            if tmpYoperated[i, j]
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                tmpXoperated[i, j] = true
                            else
                                r = rand(0:1)
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                if r == 0
                                    tmpXoperated[i, j] = true
                                else
                                    tmpYoperated[i, j] = true
                                end
                            end
                        end
                    end
                end
            end
        end
        Xoperated = tmpXoperated
        Yoperated = tmpYoperated
        return one_body_gates
    end


    circuit = QCircuit()
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    for i in 0:(depth-1)
        present_config = i % 8 + 1
        # println("present config is $present_config")
        config, configsites = configs[present_config]
        fill!(CZoperated, false)
        for item in configsites
            CZoperated[item...] = true
        end
        one_body_gates = __generate_one_body_gates()
        for item in one_body_gates
            push!(circuit, item)
        end
        for item in config
            push!(circuit, item)
        end
        # append!(circuit, one_body_gates)
        # append!(circuit, config)
    end
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    return circuit
end

google_circuit (generic function with 1 method)

In [9]:
m = 5
n = 5
depth = 8

circuit = google_circuit(m, n, depth)
println("number of gate operations $(length(circuit)).")

number of gate operations 170.


### Gate fusion technique-absorbing onebody gate into twobody gate

In [10]:
circuit = fuse_gate(circuit)
println("number of gate operations after gate fusion $(length(circuit)).")

number of gate operations after gate fusion 40.


### Brute-force simulator

In [11]:
state = statevector(ComplexF32, m * n)

# generating random basis
basis = [rand(0:1, m*n) for _ in 1:5]
apply!(circuit, state)

amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float64},1}:
 -0.044194110603886526 - 1.0167355691987394im
   0.13114699981219682 + 0.1291163541142298im
    0.4809091970882192 + 0.8806872280795137im
   0.08427169636686552 + 0.468615424992925im
    0.7929494591848093 - 0.08473197156274086im

### Matrix Product States Simulator

In [12]:
state = qstate(m * n)

apply!(circuit, state)
amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float64},1}:
 -0.04419417382416205 - 1.0167354345603952im
   0.1311468481593994 + 0.12911649948891013im
   0.4809089780164496 + 0.8806868911043519im
  0.08427184815940565 + 0.4686152816976356im
   0.7929490209613957 - 0.084731805214442im

### Schodinger-Feymann simulator

In [13]:
half_position = div(m * n, 2)
sf_circuit = SFQCircuit(ComplexF32, collect(1:half_position), collect(half_position+1:m*n), circuit, verbose=2)
amps = amplitudes(state, basis, scaling=sqrt(2))

a wedge (12, 13), (12, 17) is found, with 4 branches.


5-element Array{Complex{Float64},1}:
 -0.04419417382416205 - 1.0167354345603952im
   0.1311468481593994 + 0.12911649948891013im
   0.4809089780164496 + 0.8806868911043519im
  0.08427184815940565 + 0.4686152816976356im
   0.7929490209613957 - 0.084731805214442im

### PEPS simulator

Specifying the graph connectivity

In [14]:
connectivity = square_connectivity(m, n)
println(connectivity)

[(1, 2), (2, 3), (3, 4), (4, 5), (6, 7), (7, 8), (8, 9), (9, 10), (11, 12), (12, 13), (13, 14), (14, 15), (16, 17), (17, 18), (18, 19), (19, 20), (21, 22), (22, 23), (23, 24), (24, 25), (1, 6), (2, 7), (3, 8), (4, 9), (5, 10), (6, 11), (7, 12), (8, 13), (9, 14), (10, 15), (11, 16), (12, 17), (13, 18), (14, 19), (15, 20), (16, 21), (17, 22), (18, 23), (19, 24), (20, 25)]


In [15]:
gf_state = GraphState(ComplexF32, connectivity)

GraphState{Complex{Float32}}({25, 40} undirected simple Int64 graph, IndexedTensor{Complex{Float32},N} where N[IndexedTensor{Complex{Float32},3}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (1, 2, 3)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (4, 2, 5, 6)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (7, 5, 8, 9)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (10, 8, 11, 12)), IndexedTensor{Complex{Float32},3}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (13, 11, 14)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (15, 3, 16, 17)), IndexedTensor{Complex{Float32},5}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (18, 6, 16, 19, 20)), IndexedTensor{Complex{Float32},5}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (21, 9, 19, 22, 23)), IndexedTensor{Complex{Float32},5}(Complex{Float32}[1.0f0 + 0.0f0

In [16]:
apply!(circuit, gf_state)

amps = amplitudes(gf_state, basis, scaling=sqrt(2))

5-element Array{Complex{Float32},1}:
  -0.044193f0 - 1.016736f0im
 0.13114789f0 + 0.12911728f0im
 0.48090953f0 + 0.8806894f0im
 0.08427144f0 + 0.46861586f0im
  0.7929515f0 - 0.08473185f0im

Directly computing the amplitudes without store the final state, which maybe faster. Moreover, to get the best performance from the PEPS simulator, one needs to understand more about the problem to feed other keyword arguments into the function

In [17]:
amps = graph_amplitudes(ComplexF32, connectivity, circuit, basis, scaling=sqrt(2))

5-element Array{Complex{Float32},1}:
 -0.044195406f0 - 1.0167356f0im
   0.13114752f0 + 0.12911782f0im
    0.4809094f0 + 0.88068926f0im
   0.08427142f0 + 0.4686155f0im
   0.79295146f0 - 0.08473234f0im

## Part 3: Simulation of variational quantum circuit
Only brute-force and MPS simulator support automatic differentiation of variational quantum circuit currently

### Specifying a parameteric quantum gate-Variable wrapper

In [18]:
using Meteor.Diff

# This is a normal quantum gate
gate1 = RxGate(1, 0.5)
println("number of parameters $(nparameters(gate1)).")

# This is a variational quantum gate
gate2 = RxGate(1, Variable(0.5))
println("number of parameters $(nparameters(gate2)).")

# This is a variational two qubit gate
gate3 = CRxGate((1, 3), Variable(0.5))
println("number of parameters $(nparameters(gate3)).")

gate4 = CRxGate(control=1, target=3, theta=Variable(0.5))
println("number of parameters $(nparameters(gate4)).")

number of parameters 0.
number of parameters 1.
number of parameters 1.
number of parameters 1.


### A parameteric quantum circuit is just a quantum circuit with some parameteric quantum gates

In [19]:
# This is a normal quantum gate
gate1 = RxGate(1, Variable(0.5))

# This is a variational quantum gate
gate2 = CNOTGate(control=1,target=1)

circuit = QCircuit([gate1, gate2])
println(nparameters(circuit))

1


### A trivial example

In [20]:
L = 4
depth = 3

circuit = QCircuit()

for d in 1:depth
  for i in 1:L
    push!(circuit, RzGate(i, Variable(randn())))
    push!(circuit, RxGate(i, Variable(randn())))
    push!(circuit, RzGate(i, Variable(randn())))
  end
  for i in 1:L-1
    push!(circuit, CNOTGate(control=i, target=i+1))
  end
end

for i in 1:L
  push!(circuit, RzGate(i, Variable(randn())))
  push!(circuit, RxGate(i, Variable(randn())))
  push!(circuit, RzGate(i, Variable(randn())))
end

println("number of parameters $(nparameters(circuit)).")

# create an initial state [0,0,0,0,0]
state = qstate(L)

# A parameteric quantum circuit can be applied onto a quantum state just as a normal quantum state
apply!(circuit, state)

number of parameters 48.


9-element Array{Tuple{Int64,Float64},1}:
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (4, 0.0)
 (2, 0.0)
 (2, 0.0)
 (4, 0.0)
 (2, 0.0)

### Applying the quantum circuit onto the quantum state to produce an output quantum state, without changing the input state

In [21]:
state = qstate(L)
output_state = circuit * state

NSymMPS{Complex{Float64}}(Array{Complex{Float64},3}[[-0.5335679425031054 - 0.21834454712294188im -0.271920417207043 + 0.6531619855584935im]

[0.12683726126171174 + 0.29037072508898165im -0.23723322108873718 + 0.10191745081538081im], [-0.5556037830510745 - 0.6612070146470077im -0.15062209587779024 - 0.0440297243078204im; 0.22209165816103055 - 0.07065670732541537im 0.04630170668289582 + 0.3216538189022573im]

[0.09422941656318698 - 0.016229415135536204im -0.28487256735995453 + 0.3616257647083322im; -0.20228605347306838 - 0.0007271495880330076im 0.07310128542225505 + 0.35586431392529744im]

[-0.020672128131356448 + 0.04565426698510679im -0.024521395499172274 + 0.029393915534989636im; 0.4470315318239571 + 0.525176806775107im -0.21652497726984377 + 0.09054955069806318im]

[-0.011483412720915617 - 0.01596311548978443im -0.006637699302884801 + 0.0633106507314689im; 0.04993184848531893 - 0.10881611008661266im -0.23406835692414785 - 0.2594652713630604im], [-0.48707505157431275 - 0.7771461250754

### Create a loss function and compute the gradient

In [22]:
using Zygote

# Supported differential operations on quantum state: *, dot, distance
loss_example(circ) = begin
  out_put = circ * state
  return real(dot(out_put, state))
end


grad = parameters(gradient(loss_example, circuit))
grad[1:5]

5-element Array{Number,1}:
  0.15797547368495865
 -0.0015854082019225307
  0.15657962878699502
  0.1579754736849586
 -0.009636262976761038

### Using state vector as backend

In [23]:
state = statevector(L)

loss_example_brute_force(circ) = begin
  out_put = circ * state
  return real(dot(out_put, state))
end


grad = parameters(gradient(loss_example_brute_force, circuit))
grad[1:5]

5-element Array{Number,1}:
  0.15797547368495857
 -0.0015854082019225585
  0.15657962878699497
  0.1579754736849586
 -0.009636262976761074

### A complete example-quantum state tomography with variational quantum circuit

In [24]:
using Flux
using Flux.Optimise
using Meteor.Model: xxz, product_state!, ground_state!

function real_variational_circuit(L::Int, depth::Int)
    circuit = QCircuit()
    for i in 1:L
        add!(circuit, RyGate(i, Variable(randn(Float64))))
    end
    for i in 1:depth
        for j in 1:(L-1)
            add!(circuit, CNOTGate((j, j+1)))
        end
        for j in 1:L
            add!(circuit, RyGate(j, Variable(randn(Float64))))
        end
    end
    return circuit
end

function xxz_ground_state(L::Int, J::Real, Jzz::Real, h::Real)
    model = xxz(L, J=J, Jzz=Jzz, h=h)
    initial_state = [0 for i in 1:L]
    for i in 1:2:L
        initial_state[i] = 1
    end
    product_state!(model, initial_state)
    r = ground_state!(model, verbose=0)
    return model.state
end

L = 10
J = 1.
Jzz = 1.5
h = 0.1

initial_state = qstate(Float64, L)
target_state = xxz_ground_state(L, J, Jzz, h)

loss(circ) = 1 - abs(dot(target_state, circ * initial_state))

circuit = real_variational_circuit(L, depth)
x0 = parameters(circuit)

println("number of parameters $(length(x0)).")
opt = ADAM(0.05)

fvalues = [loss(circuit)]
for i in 1:200
    grad = parameters(gradient(loss, circuit))
    Optimise.update!(opt, x0, grad)
    set_parameters!(x0, circuit)
    ss = loss(circuit)
    push!(fvalues, ss)
    if i % 20 == 0
        println("loss at the $i-th step is $ss.")
    end
end


┌ Warning: no observers have been set.
└ @ Meteor.Model /Users/guochu/Documents/QuantumSimulator/Meteor.jl/src/Model/Model/base.jl:147


number of parameters 40.
loss at the 20-th step is 0.2325660804287406.
loss at the 40-th step is 0.07433516788147299.
loss at the 60-th step is 0.04710322038975878.
loss at the 80-th step is 0.03475686783305276.
loss at the 100-th step is 0.03009320998687759.
loss at the 120-th step is 0.029372935450680093.
loss at the 140-th step is 0.028986024336899785.
loss at the 160-th step is 0.028727805458156452.
loss at the 180-th step is 0.02852207704519949.
loss at the 200-th step is 0.028356178724046943.
